### Automating the Hedge Ratio of Statisical Arbitrage

Time Period: 1 min 

ADF_test interval: 40 periods 

Results will be showed after first 40 mins runnings this program

In [63]:
"""Importing all the important libarires:-

pytz:  setting the timezone to Erupe/London

datetime: Getting the present time 

yahoo_fin: open source library for real time data for Stocks/Futures

Statsmodel: Performs the linear regession for obtaining the residual values 

pandas: Dataframe conversion for statsmodel as  input

concurrent_futures: getting values of two stock simulteneously (advanced threading)

yfinance: check the availbility of stock data """


import pytz
import time
import datetime as dt
from yahoo_fin import stock_info as si
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import concurrent.futures
from statsmodels.tsa.stattools import adfuller
import yfinance as yf
import json

##### Getting the  Current stock prices at every minutes 

In [64]:
#appending the next price and removing the last from 40 period window
def current_price(stock_name, args):
    """ input:-
    stock_name: name of the stock/Future
    args: list contain last 40 period prices
    """
    while True:
        tz=pytz.timezone('Europe/London')
        min = dt.datetime.now(tz = tz).minute   
        if (min%1 == 0 and (dt.datetime.now(tz=tz).second % 60) == 0):  #code will be running at every minute
            price = si.get_live_price(stock_name)
            print("\nmin_",min,":",price)
            args.append(price)
            args
            args.pop(0)
            time.sleep(1)
            return args
            


#### performing the ADF test and on residual values to get live hedge ratios 

In [71]:

def hedge_ratio(stock_name1, stock_name2):
    """ Input:-
    stock_name1
    stock_name2 """
    x=list(range(1,41))
    y=list(range(1,41))
    df=pd.DataFrame(index=list(range(1,41)))
    count=1
    while True: 
        with concurrent.futures.ThreadPoolExecutor() as executor:
            t1 = executor.submit(current_price, stock_name1,x)
            t2 = executor.submit(current_price, stock_name2,y)
            x=t1.result()
            y=t2.result()
            df["x"]=t1.result()
            df["y"]=t2.result()
       
        print(df["x"][0:40])
        print(df["y"][0:40])
        
        linear_model=ols("y ~ x", data=df).fit() #performing the linear regression
        residual_values=linear_model.resid
        ADF_result = adfuller(residual_values) # performing the ADF test 
        print("hdege ratio:", linear_model.params[1] ) 
        print('ADF Statistic:',ADF_result[0])
        diction={"hedge_ratio": linear_model.params[1],
               " ADF Statistic": ADF_result[0],
                 "index": count
                }
        json_object = json.dumps(diction, indent=4)
        with open("sample.json", "w") as outfile:
            outfile.write(json_object)
        count+=1

In [72]:
def main_prog():
    print ("enter the first stock name" ) 
    stock1= str(input())
    print ("enter the second stock name")
    stock2=str(input())
    stock1.upper()
    stock2.upper()

    flag=0
    for t in [stock1,stock2]:
        ticker = yf.Ticker(t)
        info = None
        try:
            info = ticker.info
            flag+=1
        except:
            print(f"Cannot get info of {t}, it probably does not exist")
            continue
        print(f"Data exist of {t}")
    
    if(flag==2):
        hedge_ratio(stock1,stock2)
    

In [ ]:
main_prog()

enter the first stock name
cl=f
enter the second stock name
bz=f
Data exist of cl=f
Data exist of bz=f

min_ 27 : 87.33999633789062

min_ 28 : 84.0199966430664
1      2.000000
2      3.000000
3      4.000000
4      5.000000
5      6.000000
6      7.000000
7      8.000000
8      9.000000
9     10.000000
10    11.000000
11    12.000000
12    13.000000
13    14.000000
14    15.000000
15    16.000000
16    17.000000
17    18.000000
18    19.000000
19    20.000000
20    21.000000
21    22.000000
22    23.000000
23    24.000000
24    25.000000
25    26.000000
26    27.000000
27    28.000000
28    29.000000
29    30.000000
30    31.000000
31    32.000000
32    33.000000
33    34.000000
34    35.000000
35    36.000000
36    37.000000
37    38.000000
38    39.000000
39    40.000000
40    84.019997
Name: x, dtype: float64
1      2.000000
2      3.000000
3      4.000000
4      5.000000
5      6.000000
6      7.000000
7      8.000000
8      9.000000
9     10.000000
10    11.000000
11    12.000000


In [61]:
dict

dict